In [ ]:
#based off of this tutorial: http://stanford.edu/~mgorkove/cgi-bin/rpython_tutorials/Scraping_a_Webpage_Rendered_by_Javascript_Using_Python.php
#one year of data at a time -- often get "OSError: [Errno 24] Too many open files" if I try doing all at once 

from selenium import webdriver
import time
from bs4 import BeautifulSoup
import numpy as np
from datetime import date, timedelta
import pandas as pd

saveIt = True

def find_substring(substring, string):
    """ 
    Returns list of indices where substring begins in string

    >>> find_substring('me', "The cat says meow, meow")
    [13, 19]
    """
    indices = []
    index = -1  # Begin at -1 so index + 1 is 0
    while True:
        # Find next index of substring, by starting search from index + 1
        index = string.find(substring, index + 1)
        if index == -1:  
            break  # All occurrences have been found
        indices.append(index)
    return indices

#first, get list of dates to use in URLs
startYear = 2018 #change this to the year you want
finalYear = startYear

dates = []
for year in range(startYear,finalYear+1):

    startDate = str(year)+'-01-01' #YYYY-MM-DD
    finalDate = str(year)+'-03-31'

    d1 = date(int(startDate[0:4]), int(startDate[5:7]), int(startDate[8:10]))  # start date
    d2 = date(int(finalDate[0:4]), int(finalDate[6]), int(finalDate[8:10]))  # end date

    delta = d2 - d1
    dates = dates+[str(d1+timedelta(i)) for i in range(delta.days+1)] #list of dates (strings) during time range of interest
  
#now, get list of urls
urls = []
url_base = 'https://www.avalanche.ca/forecasts/archives/sea-to-sky/'
for day in dates:
    urls.append(url_base+day)

#loop through each URL and scrape 
columns = ['Below Treeline','Treeline','Above Treeline']
index = dates
riskStr = pd.DataFrame(index=index,columns = columns)
risk = pd.DataFrame(index=index,columns = columns)
for url in urls:
    
    day = url[-10:]
    print(day)
    
    options = webdriver.ChromeOptions();
    options.add_argument('headless')
    #self.driver = webdriver.Chrome(options=options)
    browser = webdriver.Chrome('/Users/samanderson/Downloads/chromedriver',options=options) #since I'm using Chrome -- also had to download 'chromedriver.exe'
    browser.get(url)
    time.sleep(8) #have to wait while the page loads -- takes a hot minute since it's a lot of javascript animations etc

    innerHTML = browser.execute_script("return document.body.innerHTML") #returns the inner HTML as a string
    html = BeautifulSoup(innerHTML,'html.parser') #makes it prettier
    webtxt = html.get_text() #just want the text for easy access to info

    inds1Low = find_substring('1 - Low',webtxt) #indices of wherever '1 - Low' appears
    inds2Mod = find_substring('2 - Moderate',webtxt) #etc
    inds3Con = find_substring('3 - Considerable',webtxt)
    inds4High = find_substring('4 - High',webtxt)

    #we want to find three smallest indices, since the first three mentions of rating are for the current day's forecast
    indsRatings = inds1Low + inds2Mod + inds3Con + inds4High 
    indsRatings.sort()
    firstThreeInds = indsRatings[0:3]
    firstThreeRatingsStr = []
    firstThreeRatings = []

    for ind in firstThreeInds: #this is bulky but it gets it done -- finds the three ratings for current day's below treeline/treeline/above treeline
    
        try:
            inds1Low.index(ind)
            firstThreeRatingsStr.append('1 - Low')
            firstThreeRatings.append(1)
        except:
            a = 1 #do nothing
        
        try:
            inds2Mod.index(ind)
            firstThreeRatingsStr.append('2 - Moderate')
            firstThreeRatings.append(2)
        except:
            a = 1 #do nothing

        try:
            inds3Con.index(ind)
            firstThreeRatingsStr.append('3 - Considerable')
            firstThreeRatings.append(3)
        except:
            a = 1 #do nothing
        
        try:
            inds4High.index(ind)
            firstThreeRatingsStr.append('4 - High')
            firstThreeRatings.append(4)
        except:
            a = 1 #do nothing

    browser.close()
    riskStr.loc[day]=firstThreeRatingsStr
    risk.loc[day]=firstThreeRatings

if saveIt is True:
    risk.to_csv('/Users/samanderson/avalanche_rating_Jan_to_Mar_'+str(startYear))
    riskStr.to_csv('/Users/samanderson/avalanche_rating_string_Jan_to_Mar_'+str(startYear))

In [ ]:
#This opens each year's risk values and concatenates them into one csv file

import pandas as pd

saveIt = True #if you want to save the concatenated file with all years

startYear = 2012
finalYear = 2017

csv_name_base = '/Users/samanderson/avalanche_rating_Jan_to_Mar_'

data2012 = pd.read_csv(csv_name_base+str(2012),index_col='Unnamed: 0')
data2013 = pd.read_csv(csv_name_base+str(2013),index_col='Unnamed: 0')
data2014 = pd.read_csv(csv_name_base+str(2014),index_col='Unnamed: 0')
data2015 = pd.read_csv(csv_name_base+str(2015),index_col='Unnamed: 0')
data2016 = pd.read_csv(csv_name_base+str(2016),index_col='Unnamed: 0')
data2017 = pd.read_csv(csv_name_base+str(2017),index_col='Unnamed: 0')

data = [data2012,data2013,data2014,data2015,data2016,data2017]

risk_all_years = pd.concat(data)

if saveIt is True:
    risk_all_years.to_csv('/Users/samanderson/avalanche_rating_Jan_to_Mar_2012_to_2017.csv')